# RLHF Canary - VS Code + Colab Development

This notebook is optimized for local VS Code development with Colab GPU runtime.

## Setup

1. **Install VS Code extension**: Extensions → search "Google Colab" → Install
2. **Open this notebook** in VS Code
3. **Select Colab kernel**: Click "Select Kernel" (top right) → "Colab" → T4 GPU
4. **Sign in** with your Google account when prompted

## Workflow

1. Edit code locally
2. Commit and push to your branch
3. Run cell 3 to pull latest changes
4. Run cell 4 to reinstall
5. Test on GPU

In [ ]:
# Cell 2: GPU verification
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("No GPU - change runtime: Runtime > Change runtime type > T4 GPU")

In [ ]:
# Cell 3: Clone or pull repo
import os

REPO_URL = "https://github.com/mmcmanus1/rlhf-canary.git"
BRANCH = "main"  # Change to your branch, e.g. "mmcmanus1/colab-local-setup"

if os.path.exists("rlhf-canary"):
    %cd rlhf-canary
    !git fetch origin
    !git checkout {BRANCH}
    !git pull origin {BRANCH}
    print(f"\nPulled latest from {BRANCH}")
else:
    !git clone -b {BRANCH} {REPO_URL}
    %cd rlhf-canary
    print(f"\nCloned {BRANCH}")

!git log -1 --oneline

In [ ]:
# Cell 4: Install dependencies
!pip install -q -e .
!pip install -q bitsandbytes
print("Installation complete")

In [ ]:
# Cell 5: Verify installation
!python -m canary.cli env

## Run Tests

Use the cells below to run canary tests on GPU.

In [ ]:
# Cell 6: Run smoke test (~5-10 min on T4)
!python -m canary.cli run configs/dpo_smoke.yaml -o ./canary_output

In [ ]:
# Cell 7: View results
import json
from pathlib import Path

metrics_files = sorted(Path('./canary_output').rglob('metrics.json'), key=lambda p: p.stat().st_mtime)
if metrics_files:
    latest = metrics_files[-1]
    print(f"Latest run: {latest.parent.name}")
    
    with open(latest) as f:
        m = json.load(f)
    
    print(f"\nPerformance:")
    print(f"  Step time (mean): {m['perf']['step_time']['mean']:.4f}s")
    print(f"  Tokens/sec: {m['perf']['approx_tokens_per_sec']:.0f}")
    print(f"  Peak memory: {m['perf']['max_mem_mb']:.0f}MB")
    
    print(f"\nStability:")
    print(f"  NaN steps: {m['stability']['nan_steps']}")
    print(f"  Inf steps: {m['stability']['inf_steps']}")
else:
    print("No metrics found. Run cell 6 first.")

## Compare Runs

Save a baseline and compare subsequent runs.

In [ ]:
# Cell 8: Save current run as baseline
from pathlib import Path
import shutil

metrics_files = sorted(Path('./canary_output').rglob('metrics.json'), key=lambda p: p.stat().st_mtime)
if metrics_files:
    Path('baselines').mkdir(exist_ok=True)
    shutil.copy(metrics_files[-1], 'baselines/baseline.json')
    print("Saved baselines/baseline.json")
else:
    print("No metrics to save. Run cell 6 first.")

In [ ]:
# Cell 9: Compare latest run to baseline
from pathlib import Path

metrics_files = sorted(Path('./canary_output').rglob('metrics.json'), key=lambda p: p.stat().st_mtime)
if metrics_files and Path('baselines/baseline.json').exists():
    latest = metrics_files[-1]
    !python -m canary.cli compare {latest} baselines/baseline.json --threshold-tier smoke
else:
    print("Need both a baseline (cell 8) and a new run (cell 6) to compare.")